# Train

In [1]:
import pickle
import numpy as np
from sklearn.utils import shuffle

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam

Using TensorFlow backend.


## 1 Reload data

In [2]:
def reload_data(pickle_file):
    print('reload: ', pickle_file)
    with open(pickle_file, 'rb') as f:
        pickle_data = pickle.load(f)
        X_train = pickle_data['X_train']
        y_train = pickle_data['y_train']
        del pickle_data  # Free up memory
    return X_train, y_train

X_train, y_train = reload_data('./pre-data.pickle')
X_train, y_train = shuffle(X_train, y_train)
print('X_train shape: ', X_train.shape, 'y_train shape: ',y_train.shape)

reload:  ./pre-data.pickle
X_train shape:  (24108, 80, 80, 3) y_train shape:  (24108,)


## 2 Model architecture

In [3]:
nvidia = Sequential()
nvidia.add(Lambda(lambda x: x/255. - 0.5, input_shape=(80, 80, 3)))
nvidia.add(Cropping2D(cropping=((35, 13), (0, 0))))
nvidia.add(Convolution2D(24, 3, 3, subsample=(2, 2), activation='relu'))
nvidia.add(Convolution2D(36, 3, 3, subsample=(2, 2), activation='relu'))
nvidia.add(Convolution2D(48, 3, 3, activation='relu'))
nvidia.add(Convolution2D(64, 3, 3, activation='relu'))
nvidia.add(Convolution2D(64, 3, 3, activation='relu'))
nvidia.add(Dropout(0.5))
nvidia.add(Flatten())
nvidia.add(Dense(100))
nvidia.add(Dense(50))
nvidia.add(Dense(10))
nvidia.add(Dense(1))

## 3 Train the model

In [4]:
# Hyperparameters
LEARNING_RATE = 1e-4
EPOCHS = 5

# Training
nvidia.compile(loss='mse', optimizer=Adam(LEARNING_RATE))
nvidia.fit(X_train, y_train, validation_split=0.2, 
           shuffle=True, nb_epoch=EPOCHS)
nvidia.save('model.h5')

Train on 19286 samples, validate on 4822 samples
Epoch 1/5
19286/19286 [==============================] - 130s - loss: 0.0131 - val_loss: 0.0111
Epoch 2/5
19286/19286 [==============================] - 10s - loss: 0.0110 - val_loss: 0.0108
Epoch 3/5
19286/19286 [==============================] - 12s - loss: 0.0105 - val_loss: 0.0104
Epoch 4/5
19286/19286 [==============================] - 12s - loss: 0.0102 - val_loss: 0.0100
Epoch 5/5
19286/19286 [==============================] - 12s - loss: 0.0099 - val_loss: 0.0102


## 4 Train the model with generator

In [13]:
def generator(samples, batch_size):
    ns = len(samples)
    while 1:
        samples = shuffle(samples)
        for offset in range(0, ns, batch_size):
            batch_samples = samples[offset:offset + batch_size]
            images = []
            angles = []
            for sample in batch_samples:
                center_path = './sample_data/data/IMG/' + sample[0].split('/')[-1]
                center_img = cv2.imread(center_path)
                center_img = cv2.resize(center_img, None, fx=0.25, fy=0.5)
                center_img = cv2.cvtColor(center_img, cv2.COLOR_BGR2RGB)
                center_angle = float(line[3])
                images.append(center_img)
                angles.append(center_angle)
                images.append(cv2.flip(center_img, 1))
                angles.append(-center_angle)

                left_path = './sample_data/data/IMG/' + sample[1].split('/')[-1]
                left_img = cv2.imread(left_path)
                left_img = cv2.resize(left_img, None, fx=0.25, fy=0.5)
                left_img = cv2.cvtColor(left_img, cv2.COLOR_BGR2RGB)
                left_angle = float(line[3]) + 0.20
                images.append(left_img)
                angles.append(left_angle)

                right_path = './sample_data/data/IMG/' + sample[2].split('/')[-1]
                right_img = cv2.imread(right_path)
                right_img = cv2.resize(right_img, None, fx=0.25, fy=0.5)
                right_img = cv2.cvtColor(right_img, cv2.COLOR_BGR2RGB)
                right_angle = float(line[3]) - 0.25
                images.append(right_img)
                angles.append(right_angle)
                
            X_train = np.array(images)
            y_train = np.array(angles)
            yield shuffle(X_train, y_train)
            
train_generator = generator(train_lines, batch_size=128)
valida_generator = generator(valida_lines, batch_size=128)

In [ ]:
# Hyperparameters
LEARNING_RATE = 1e-4
EPOCHS = 7

# Training
nvidia.compile(loss='mse', optimizer=Adam(LEARNING_RATE))
nvidia.fit_generator(train_generator, 
                     samples_per_epoch=len(4*train_lines),
                     validation_data=valida_generator, 
                     nb_val_samples=len(4*valida_lines), 
                     nb_epoch=EPOCHS)
nvidia.save('model.h5')